In [ ]:
import pandas as pd
import numpy as np
import cv2
import os

In [ ]:
home_dir = os.getenv("HOME")
raw_data = pd.read_csv(home_dir + '/Desktop/DataPacket/data.csv')

In [ ]:
raw_data.tail()

In [ ]:
img_input = []
output = raw_data["steering_angle"].values

In [ ]:
h = 240
w = 320
img_num = len(raw_data) + 1  # Because csv lenght start with 0

for i in range(1, img_num):
    img = cv2.imread(home_dir + "/Desktop/DataPacket/Images/"+str(i)+".jpg", 3)
    img = cv2.resize(img, (w, h))
    
    img_input.append(img)

In [ ]:
img_input = np.asarray(img_input).astype(np.float32) / 255.0

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(img_input, output, test_size=0.2)

del img_input
del output

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, Cropping2D, Lambda
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers.core import Activation

In [ ]:
X_train.shape

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 2, kernel_size = (3,3),padding = 'same',activation ='relu', input_shape = (h, w, 3)))
model.add(Conv2D(filters = 4, kernel_size = (3,3),padding = 'same',activation ='relu'))
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 24, kernel_size = (7,7),padding = 'same',activation ='relu'))

model.add(Flatten())

model.add(Dense(50, activation='relu'))
model.add(Dense(28, activation='relu'))
model.add(Dense(10, activation='relu'))

model.add(Dense(1, activation = "sigmoid"))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


In [ ]:
hs = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=64, epochs=5)

In [ ]:
import rospkg
import rospy

rospack = rospkg.RosPack()
package_path = rospack.get_path('angelshark_deeplearning')

In [ ]:
import json
import matplotlib.pylab as plt
import random
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

# Save model weights and json.
model.save_weights(package_path + '/model/model.h5')
model_json = model.to_json()
with open(package_path + '/model/model.json', 'w') as outfile:
    json.dump(model_json, outfile)

# Train and validation loss chart
print(hs.history.keys())

plt.plot(hs.history['loss'])
plt.plot(hs.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()
